In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2893.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2765.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2733.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_1037.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2961.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_1373.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_271.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2418.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2055.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2127.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2907.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_123.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_1782.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_2181.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_562.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/SeaLake_1452.tif
/kaggle/input/all-band-eurosat/tif/SeaLake/

**CHANNEL BASED MODEL**

In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 80.4 MB/s eta 0:00:00:00:0100:01


**BASE PAPER MODEL-SWIR-NIR-RED**

In [7]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# ----------- Parameters -----------
data_dir = "/kaggle/input/all-band-eurosat/tif"
img_size = (64, 64)
selected_channels = [12, 7, 3]  # SWIR, NIR, Red
batch_size = 64
epochs = 100

# ----------- Load Data -----------
def load_tif_images(base_dir, img_size=(64, 64), selected_channels=[12, 7, 3]):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    
    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    img = src.read(selected_channels)  # shape: (3, H, W)
                    img = np.moveaxis(img, 0, -1)      # shape: (H, W, 3)
                    img = img.astype(np.float32) / 255.0
                    img = tf.image.resize(img, img_size).numpy()
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

X, y, class_names = load_tif_images(data_dir, img_size, selected_channels)

# ----------- Encode Labels -----------
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# ----------- Train-Test Split -----------
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)

# ----------- Dataset API -----------
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# ----------- CNN Model -----------
def create_custom_cnn(num_classes):
    inputs = tf.keras.Input(shape=(64, 64, 3))

    # Conv Block 1
    x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='gelu')(inputs)
    x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Conv Block 2
    x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Conv Block 3
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Conv Block 4
    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Conv Block 5
    x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Fully Connected Layers
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation='gelu')(x)
    x = tf.keras.layers.Dense(256, activation='gelu')(x)
    x = tf.keras.layers.Dense(128, activation='gelu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------- Train Model -----------
model = create_custom_cnn(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# ----------- Evaluation -----------
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 64, 64, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 64, 64, 16)          │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 32, 32, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 4, 4, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 4, 4, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 2, 2, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 1,869,082 (7.13 MB)

 Trainable params: 1,869,082 (7.13 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 19s 27ms/step - accuracy: 0.3889 - loss: 1.6388 - val_accuracy: 0.7407 - val_loss: 0.7357
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.7700 - loss: 0.6481 - val_accuracy: 0.7891 - val_loss: 0.5863
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8384 - loss: 0.4737 - val_accuracy: 0.8176 - val_loss: 0.5363
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8738 - loss: 0.3810 - val_accuracy: 0.9020 - val_loss: 0.3070
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8982 - loss: 0.3132 - val_accuracy: 0.8931 - val_loss: 0.3446
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9003 - loss: 0.2951 - val_accuracy: 0.8946 - val_loss: 0.3166
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9141 - loss: 0.2566 - val_accuracy: 0.9026 - val_loss: 0.3061
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9294 - loss: 0.2091 -

**BASE PAPER METHOD-NIR-RED-GREEM**

In [8]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# ----------- Parameters -----------
data_dir = "/kaggle/input/all-band-eurosat/tif"
img_size = (64, 64)
selected_channels = [7, 3, 2]  
batch_size = 64
epochs = 100

# ----------- Load Data -----------
def load_tif_images(base_dir, img_size=(64, 64), selected_channels=[7, 3, 2]):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    
    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    img = src.read(selected_channels)  # shape: (3, H, W)
                    img = np.moveaxis(img, 0, -1)      # shape: (H, W, 3)
                    img = img.astype(np.float32) / 255.0
                    img = tf.image.resize(img, img_size).numpy()
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

X, y, class_names = load_tif_images(data_dir, img_size, selected_channels)

# ----------- Encode Labels -----------
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# ----------- Train-Test Split -----------
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)

# ----------- Dataset API -----------
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# ----------- CNN Model -----------
def create_custom_cnn(num_classes):
    inputs = tf.keras.Input(shape=(64, 64, 3))

    # Conv Block 1
    x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='gelu')(inputs)
    x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Conv Block 2
    x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Conv Block 3
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Conv Block 4
    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Conv Block 5
    x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    # Fully Connected Layers
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation='gelu')(x)
    x = tf.keras.layers.Dense(256, activation='gelu')(x)
    x = tf.keras.layers.Dense(128, activation='gelu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------- Train Model -----------
model = create_custom_cnn(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# ----------- Evaluation -----------
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 64, 64, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 64, 64, 16)          │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 32, 32, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 4, 4, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 4, 4, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 2, 2, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 1,869,082 (7.13 MB)

 Trainable params: 1,869,082 (7.13 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.3353 - loss: 1.7723 - val_accuracy: 0.7019 - val_loss: 0.8244
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.7264 - loss: 0.7554 - val_accuracy: 0.7928 - val_loss: 0.5808
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.7897 - loss: 0.5876 - val_accuracy: 0.8317 - val_loss: 0.4769
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8578 - loss: 0.4124 - val_accuracy: 0.8450 - val_loss: 0.4346
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8819 - loss: 0.3461 - val_accuracy: 0.8648 - val_loss: 0.3781
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8826 - loss: 0.3357 - val_accuracy: 0.8781 - val_loss: 0.3640
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9141 - loss: 0.2527 - val_accuracy: 0.8887 - val_loss: 0.3279
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9167 - loss: 0.2325 -

**BASE PAPER --NDVI-NDBI-NDWI**

In [4]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# ----------- Parameters -----------
data_dir = "/kaggle/input/all-band-eurosat/tif"
img_size = (64, 64)
batch_size = 64
epochs = 100

# ----------- Load Data with NDVI, NDBI, NDWI -----------
def load_indices_images(base_dir, img_size=(64, 64)):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])

    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    nir = src.read(7).astype(np.float32)
                    red = src.read(3).astype(np.float32)
                    green = src.read(2).astype(np.float32)
                    swir = src.read(12).astype(np.float32)

                    # Compute indices with small value added to denominator to avoid division by zero
                    ndvi = (nir - red) / (nir + red + 1e-6)
                    ndbi = (swir - nir) / (swir + nir + 1e-6)
                    ndwi = (green - nir) / (green + nir + 1e-6)

                    # Stack into a 3-channel image
                    img = np.stack([ndvi, ndbi, ndwi], axis=-1)
                    img = tf.image.resize(img, img_size).numpy()
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

X, y, class_names = load_indices_images(data_dir, img_size)

# ----------- Encode Labels -----------
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# ----------- Train-Test Split -----------
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)

# ----------- Dataset API -----------
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# ----------- CNN Model (unchanged) -----------
def create_custom_cnn(num_classes):
    inputs = tf.keras.Input(shape=(64, 64, 3))

    x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='gelu')(inputs)
    x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='gelu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation='gelu')(x)
    x = tf.keras.layers.Dense(256, activation='gelu')(x)
    x = tf.keras.layers.Dense(128, activation='gelu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------- Train Model -----------
model = create_custom_cnn(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# ----------- Evaluation -----------
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 64, 64, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 64, 64, 16)          │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 32, 32, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_24 (MaxPooling2D)      │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_25 (MaxPooling2D)      │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_30 (Conv2D)                   │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_26 (MaxPooling2D)      │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_31 (Conv2D)                   │ (None, 4, 4, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_32 (Conv2D)                   │ (None, 4, 4, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_27 (MaxPooling2D)      │ (None, 2, 2, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 1,869,082 (7.13 MB)

 Trainable params: 1,869,082 (7.13 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - accuracy: 0.3486 - loss: 1.7083 - val_accuracy: 0.6374 - val_loss: 0.9792
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6666 - loss: 0.8689 - val_accuracy: 0.7346 - val_loss: 0.7178
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7721 - loss: 0.6280 - val_accuracy: 0.7724 - val_loss: 0.6175
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8197 - loss: 0.4968 - val_accuracy: 0.7843 - val_loss: 0.6178
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8450 - loss: 0.4292 - val_accuracy: 0.8278 - val_loss: 0.4862
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8654 - loss: 0.3708 - val_accuracy: 0.8422 - val_loss: 0.4263
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8834 - loss: 0.3264 - val_accuracy: 0.8600 - val_loss: 0.3935
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9021 - loss: 0.2750 -

***SATRAT OF PROPOSED METHODOLOGY***

**PROPOSED --NIR-RED--GREEN**

In [12]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# ----------- Parameters -----------
data_dir = "/kaggle/input/all-band-eurosat/tif"
img_size = (64, 64)
selected_channels = [7, 3, 2] 
batch_size = 64
epochs = 100

# ----------- Load Data -----------
def load_tif_images(base_dir, img_size=(64, 64), selected_channels=[7, 3, 2]):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    
    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    img = src.read(selected_channels)  # shape: (3, H, W)
                    img = np.moveaxis(img, 0, -1)      # shape: (H, W, 3)
                    img = img.astype(np.float32) / 255.0
                    img = tf.image.resize(img, img_size).numpy()
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

X, y, class_names = load_tif_images(data_dir, img_size, selected_channels)

# ----------- Encode Labels -----------
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# ----------- Train-Test Split -----------
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)

# ----------- Dataset API -----------
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# ----------- CNN Model: Color-Specific -----------
def create_color_specific_model(num_classes):
    """CNN Model with Color-Specific Convolutions"""
    inputs = tf.keras.Input(shape=(64, 64, 3))
    
    channel1 = inputs[:, :, :, 0:1]  # First selected channel (e.g., 7)
    channel2 = inputs[:, :, :, 1:2]  # Second selected channel (e.g., 3)
    channel3 = inputs[:, :, :, 2:3]  # Third selected channel (e.g., 2)
    
    red_conv = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(channel1)
    red_pool = tf.keras.layers.MaxPooling2D(2,2)(red_conv)
    
    green_conv = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(channel2)
    green_pool = tf.keras.layers.MaxPooling2D(2,2)(green_conv)
    
    blue_conv = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(channel3)
    blue_pool = tf.keras.layers.MaxPooling2D(2,2)(blue_conv)
    
    combined = tf.keras.layers.Concatenate()([red_pool, green_pool, blue_pool])
    
    conv1 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same')(combined)
    pool1 = tf.keras.layers.MaxPooling2D(2,2)(conv1)
    
    conv2 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(pool1)
    pool2 = tf.keras.layers.MaxPooling2D(2,2)(conv2)
    
    flatten = tf.keras.layers.Flatten()(pool2)
    dense1 = tf.keras.layers.Dense(128, activation='relu')(flatten)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(dense1)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.AdamW(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------- Train Model -----------
model = create_color_specific_model(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# ----------- Evaluation -----------
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_3 (GetItem)      │ (None, 64, 64, 1)      │              0 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_4 (GetItem)      │ (None, 64, 64, 1)      │              0 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_5 (GetItem)      │ (None, 64, 64, 1)      │              0 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_43 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_3[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_44 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_4[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_45 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_5[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_28          │ (None, 32, 32, 32)     │              0 │ conv2d_43[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_29          │ (None, 32, 32, 32)     │              0 │ conv2d_44[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_30          │ (None, 32, 32, 32)     │              0 │ conv2d_45[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 32, 32, 96)     │              0 │ max_pooling2d_28[0][0… │
│ (Concatenate)             │                        │                │ max_pooling2d_29[0][0… │
│                           │                        │                │ max_pooling2d_30[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_46 (Conv2D)        │ (None, 32, 32, 64)     │         55,360 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_31          │ (None, 16, 16, 64)     │              0 │ conv2d_46[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_47 (Conv2D)        │ (None, 16, 16, 128)    │         73,856 │ max_pooling2d_31[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_32          │ (None, 8, 8, 128)      │              0 │ conv2d_47[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_4 (Flatten)  

 Total params: 1,180,170 (4.50 MB)

 Trainable params: 1,180,170 (4.50 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.4910 - loss: 1.4040 - val_accuracy: 0.6841 - val_loss: 0.8374
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7868 - loss: 0.6036 - val_accuracy: 0.8574 - val_loss: 0.4087
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8363 - loss: 0.4620 - val_accuracy: 0.8859 - val_loss: 0.3239
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8762 - loss: 0.3459 - val_accuracy: 0.8804 - val_loss: 0.3443
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9025 - loss: 0.2746 - val_accuracy: 0.8804 - val_loss: 0.3340
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9165 - loss: 0.2421 - val_accuracy: 0.9063 - val_loss: 0.2842
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9267 - loss: 0.2150 - val_accuracy: 0.9031 - val_loss: 0.2910
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9336 - loss: 0.1823 - val_

**PROPOSED SE BASED NIR-RED-GREEN MODEL**

In [8]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# --- Parameters ---
data_dir = "/kaggle/input/all-band-eurosat/tif"
img_size = (64, 64)
selected_channels = [7, 3, 2]
batch_size = 64
epochs = 100

# --- Load Data ---
def load_tif_images(base_dir, img_size=(64, 64), selected_channels=[7, 3, 2]):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    
    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    img = src.read(selected_channels)  # (3, H, W)
                    img = np.moveaxis(img, 0, -1)      # (H, W, 3)
                    img = img.astype(np.float32) / 255.0
                    img = tf.image.resize(img, img_size).numpy()
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

X, y, class_names = load_tif_images(data_dir, img_size, selected_channels)

# --- Encode Labels ---
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# --- Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)

# --- TF Datasets ---
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# --- SE Block ---
def se_block(input_tensor, reduction=8):
    filters = input_tensor.shape[-1]
    se = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
    se = tf.keras.layers.Dense(filters // reduction, activation='relu')(se)
    se = tf.keras.layers.Dense(filters, activation='sigmoid')(se)
    se = tf.keras.layers.Reshape((1, 1, filters))(se)
    return tf.keras.layers.Multiply()([input_tensor, se])

# --- Model ---
def create_color_specific_model(num_classes):
    inputs = tf.keras.Input(shape=(64, 64, 3))
    
    channel1 = inputs[:, :, :, 0:1]
    channel2 = inputs[:, :, :, 1:2]
    channel3 = inputs[:, :, :, 2:3]
    
    red_conv = tf.keras.layers.Conv2D(24, (3,3), activation='relu', padding='same')(channel1)
    red_se = se_block(red_conv)
    red_pool = tf.keras.layers.MaxPooling2D(2,2)(red_se)
    
    green_conv = tf.keras.layers.Conv2D(24, (3,3), activation='relu', padding='same')(channel2)
    green_se = se_block(green_conv)
    green_pool = tf.keras.layers.MaxPooling2D(2,2)(green_se)
    
    blue_conv = tf.keras.layers.Conv2D(24, (3,3), activation='relu', padding='same')(channel3)
    blue_se = se_block(blue_conv)
    blue_pool = tf.keras.layers.MaxPooling2D(2,2)(blue_se)
    
    combined = tf.keras.layers.Concatenate()([red_pool, green_pool, blue_pool])
    
    conv1 = tf.keras.layers.Conv2D(48, (3,3), activation='relu', padding='same')(combined)
    conv1_se = se_block(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(2,2)(conv1_se)
    
    conv2 = tf.keras.layers.Conv2D(96, (3,3), activation='relu', padding='same')(pool1)
    conv2_se = se_block(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(2,2)(conv2_se)
    
    flatten = tf.keras.layers.Flatten()(pool2)
    dense1 = tf.keras.layers.Dense(192, activation='relu')(flatten)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(dense1)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.AdamW(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- Train ---
model = create_color_specific_model(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# --- Evaluation ---
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_28 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_29 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_30 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_48 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_28[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_49 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_29[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_50 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_30[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_48[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_49[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_50[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_34 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_36 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_38 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_35 (Dense)          │ (None, 24)             │             96 │ dense_34[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_37 (Dense)          │ (None, 24)             │             96 │ dense_36[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_39 (Dense)          │ (None, 24)             │             96 │ dense_38[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_10 (Reshape)      │ (None, 1, 1, 24)       │              0 │ dense_35[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_11 (Reshape) 

 Total params: 1,258,765 (4.80 MB)

 Trainable params: 1,258,765 (4.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - accuracy: 0.5170 - loss: 1.2882 - val_accuracy: 0.7170 - val_loss: 0.7614
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7795 - loss: 0.6159 - val_accuracy: 0.7770 - val_loss: 0.6229
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8420 - loss: 0.4498 - val_accuracy: 0.8674 - val_loss: 0.3902
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8724 - loss: 0.3624 - val_accuracy: 0.8793 - val_loss: 0.3437
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8813 - loss: 0.3279 - val_accuracy: 0.8722 - val_loss: 0.3379
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9055 - loss: 0.2699 - val_accuracy: 0.8474 - val_loss: 0.4288
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9093 - loss: 0.2520 - val_accuracy: 0.8730 - val_loss: 0.3677
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9299 - loss: 0.1915 -

**PROPOSED-SWIR-NIR-RED**

In [5]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# ----------- Parameters -----------
data_dir = "/kaggle/input/all-band-eurosat/tif"
img_size = (64, 64)
selected_channels = [12, 7, 3] 
batch_size = 64
epochs = 100

# ----------- Load Data -----------
def load_tif_images(base_dir, img_size=(64, 64), selected_channels=[12, 7, 3]):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    
    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    img = src.read(selected_channels)  # shape: (3, H, W)
                    img = np.moveaxis(img, 0, -1)      # shape: (H, W, 3)
                    img = img.astype(np.float32) / 255.0
                    img = tf.image.resize(img, img_size).numpy()
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

X, y, class_names = load_tif_images(data_dir, img_size, selected_channels)

# ----------- Encode Labels -----------
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# ----------- Train-Test Split -----------
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)

# ----------- Dataset API -----------
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# ----------- CNN Model: Color-Specific -----------
def create_color_specific_model(num_classes):
    """CNN Model with Color-Specific Convolutions"""
    inputs = tf.keras.Input(shape=(64, 64, 3))
    
    channel1 = inputs[:, :, :, 0:1]  # First selected channel (e.g., 7)
    channel2 = inputs[:, :, :, 1:2]  # Second selected channel (e.g., 3)
    channel3 = inputs[:, :, :, 2:3]  # Third selected channel (e.g., 2)
    
    red_conv = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(channel1)
    red_pool = tf.keras.layers.MaxPooling2D(2,2)(red_conv)
    
    green_conv = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(channel2)
    green_pool = tf.keras.layers.MaxPooling2D(2,2)(green_conv)
    
    blue_conv = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(channel3)
    blue_pool = tf.keras.layers.MaxPooling2D(2,2)(blue_conv)
    
    combined = tf.keras.layers.Concatenate()([red_pool, green_pool, blue_pool])
    
    conv1 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same')(combined)
    pool1 = tf.keras.layers.MaxPooling2D(2,2)(conv1)
    
    conv2 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(pool1)
    pool2 = tf.keras.layers.MaxPooling2D(2,2)(conv2)
    
    flatten = tf.keras.layers.Flatten()(pool2)
    dense1 = tf.keras.layers.Dense(128, activation='relu')(flatten)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(dense1)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.AdamW(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------- Train Model -----------
model = create_color_specific_model(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# ----------- Evaluation -----------
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_19 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_20 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_21 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_33 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_19[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_34 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_20[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_35 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_21[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_28          │ (None, 32, 32, 32)     │              0 │ conv2d_33[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_29          │ (None, 32, 32, 32)     │              0 │ conv2d_34[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_30          │ (None, 32, 32, 32)     │              0 │ conv2d_35[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 32, 32, 96)     │              0 │ max_pooling2d_28[0][0… │
│ (Concatenate)             │                        │                │ max_pooling2d_29[0][0… │
│                           │                        │                │ max_pooling2d_30[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_36 (Conv2D)        │ (None, 32, 32, 64)     │         55,360 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_31          │ (None, 16, 16, 64)     │              0 │ conv2d_36[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_37 (Conv2D)        │ (None, 16, 16, 128)    │         73,856 │ max_pooling2d_31[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_32          │ (None, 8, 8, 128)      │              0 │ conv2d_37[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_3 (Flatten)  

 Total params: 1,180,170 (4.50 MB)

 Trainable params: 1,180,170 (4.50 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.5181 - loss: 1.4423 - val_accuracy: 0.7307 - val_loss: 0.7225
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7365 - loss: 0.6985 - val_accuracy: 0.8393 - val_loss: 0.4657
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8450 - loss: 0.4372 - val_accuracy: 0.8744 - val_loss: 0.3525
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8750 - loss: 0.3418 - val_accuracy: 0.8874 - val_loss: 0.3117
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8981 - loss: 0.2951 - val_accuracy: 0.8943 - val_loss: 0.3043
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9152 - loss: 0.2420 - val_accuracy: 0.8969 - val_loss: 0.2867
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9273 - loss: 0.2075 - val_accuracy: 0.8898 - val_loss: 0.3112
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9392 - loss: 0.1683 - val_ac

**PROPOSED SE BASED SWIR-NIR-RED MODEL**

In [10]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# --- Parameters ---
data_dir = "/kaggle/input/all-band-eurosat/tif"
img_size = (64, 64)
selected_channels = [12, 7, 3]
batch_size = 64
epochs = 100

# --- Load Data ---
def load_tif_images(base_dir, img_size=(64, 64), selected_channels=[12, 7, 3]):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    
    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    img = src.read(selected_channels)  # (3, H, W)
                    img = np.moveaxis(img, 0, -1)      # (H, W, 3)
                    img = img.astype(np.float32) / 255.0
                    img = tf.image.resize(img, img_size).numpy()
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

X, y, class_names = load_tif_images(data_dir, img_size, selected_channels)

# --- Encode Labels ---
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# --- Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)

# --- TF Datasets ---
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# --- SE Block ---
def se_block(input_tensor, reduction=8):
    filters = input_tensor.shape[-1]
    se = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
    se = tf.keras.layers.Dense(filters // reduction, activation='relu')(se)
    se = tf.keras.layers.Dense(filters, activation='sigmoid')(se)
    se = tf.keras.layers.Reshape((1, 1, filters))(se)
    return tf.keras.layers.Multiply()([input_tensor, se])

# --- Model ---
def create_color_specific_model(num_classes):
    inputs = tf.keras.Input(shape=(64, 64, 3))
    
    channel1 = inputs[:, :, :, 0:1]
    channel2 = inputs[:, :, :, 1:2]
    channel3 = inputs[:, :, :, 2:3]
    
    red_conv = tf.keras.layers.Conv2D(24, (3,3), activation='relu', padding='same')(channel1)
    red_se = se_block(red_conv)
    red_pool = tf.keras.layers.MaxPooling2D(2,2)(red_se)
    
    green_conv = tf.keras.layers.Conv2D(24, (3,3), activation='relu', padding='same')(channel2)
    green_se = se_block(green_conv)
    green_pool = tf.keras.layers.MaxPooling2D(2,2)(green_se)
    
    blue_conv = tf.keras.layers.Conv2D(24, (3,3), activation='relu', padding='same')(channel3)
    blue_se = se_block(blue_conv)
    blue_pool = tf.keras.layers.MaxPooling2D(2,2)(blue_se)
    
    combined = tf.keras.layers.Concatenate()([red_pool, green_pool, blue_pool])
    
    conv1 = tf.keras.layers.Conv2D(48, (3,3), activation='relu', padding='same')(combined)
    conv1_se = se_block(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(2,2)(conv1_se)
    
    conv2 = tf.keras.layers.Conv2D(96, (3,3), activation='relu', padding='same')(pool1)
    conv2_se = se_block(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(2,2)(conv2_se)
    
    flatten = tf.keras.layers.Flatten()(pool2)
    dense1 = tf.keras.layers.Dense(192, activation='relu')(flatten)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(dense1)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.AdamW(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- Train ---
model = create_color_specific_model(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# --- Evaluation ---
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_34 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_35 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_36 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_58 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_34[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_59 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_35[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_60 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_36[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_58[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_59[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_60[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_58 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_60 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_62 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_59 (Dense)          │ (None, 24)             │             96 │ dense_58[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_61 (Dense)          │ (None, 24)             │             96 │ dense_60[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_63 (Dense)          │ (None, 24)             │             96 │ dense_62[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_20 (Reshape)      │ (None, 1, 1, 24)       │              0 │ dense_59[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_21 (Reshape) 

 Total params: 1,258,765 (4.80 MB)

 Trainable params: 1,258,765 (4.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.5419 - loss: 1.2325 - val_accuracy: 0.7772 - val_loss: 0.6152
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8042 - loss: 0.5439 - val_accuracy: 0.8428 - val_loss: 0.4235
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8576 - loss: 0.4014 - val_accuracy: 0.8674 - val_loss: 0.3572
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8706 - loss: 0.3566 - val_accuracy: 0.8896 - val_loss: 0.3141
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9048 - loss: 0.2699 - val_accuracy: 0.8820 - val_loss: 0.3212
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9133 - loss: 0.2397 - val_accuracy: 0.8435 - val_loss: 0.4560
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9217 - loss: 0.2143 - val_accuracy: 0.9035 - val_loss: 0.3029
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9367 - loss: 0.1788 -

**PROSED--NDVI-NDBI-NDWI MODEL**

In [3]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# ----------- Parameters -----------
data_dir = "/kaggle/input/all-band-eurosat/tif"
img_size = (64, 64)
selected_channels = [7, 3, 2]  # NIR, Green, Blue
batch_size = 64
epochs = 100

# ----------- Load Data with NDVI, NDBI, NDWI -----------
def load_tif_images(base_dir, img_size=(64, 64), selected_channels=[7, 3, 2]):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    
    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    full_img = src.read()  # shape: (13, H, W)
                    
                    # Normalize
                    full_img = full_img.astype(np.float32) / 255.0
                    
                    # Get selected bands
                    img = full_img[selected_channels]  # shape: (3, H, W)
                    img = np.moveaxis(img, 0, -1)      # (H, W, 3)
                    
                    # Extract bands for index calculation
                    nir = full_img[7]    # Band 8 (NIR)
                    red = full_img[3]    # Band 4
                    green = full_img[2]  # Band 3
                    swir = full_img[10]  # Band 11
                    
                    # Resize all bands to match img_size
                    nir = tf.image.resize(nir[..., np.newaxis], img_size).numpy().squeeze()
                    red = tf.image.resize(red[..., np.newaxis], img_size).numpy().squeeze()
                    green = tf.image.resize(green[..., np.newaxis], img_size).numpy().squeeze()
                    swir = tf.image.resize(swir[..., np.newaxis], img_size).numpy().squeeze()
                    img = tf.image.resize(img, img_size).numpy()
                    
                    def safe_index(a, b):
                        return (a - b) / (a + b + 1e-6)
                    
                    ndvi = safe_index(nir, red)
                    ndbi = safe_index(swir, nir)
                    ndwi = safe_index(green, nir)
                    
                    # Stack indices as additional channels
                    indices = np.stack([ndvi, ndbi, ndwi], axis=-1)
                    
                    img = np.concatenate([img, indices], axis=-1)  # shape: (H, W, 6)
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

X, y, class_names = load_tif_images(data_dir, img_size, selected_channels)

# ----------- Encode Labels -----------
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# ----------- Train-Test Split -----------
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)

# ----------- Dataset API -----------
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# ----------- Updated CNN Model -----------
def create_model_with_indices(num_classes):
    inputs = tf.keras.Input(shape=(64, 64, 6))  # 3 bands + 3 indices
    
    # Split input into bands and indices
    ch1 = inputs[:, :, :, 0:1]  # NIR (band 7)
    ch2 = inputs[:, :, :, 1:2]  # GREEN (band 3)
    ch3 = inputs[:, :, :, 2:3]  # BLUE (band 2)
    ndvi = inputs[:, :, :, 3:4]
    ndbi = inputs[:, :, :, 4:5]
    ndwi = inputs[:, :, :, 5:6]

    def conv_branch(x):
        x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        return tf.keras.layers.MaxPooling2D(2, 2)(x)

    branches = [conv_branch(ch) for ch in [ch1, ch2, ch3, ndvi, ndbi, ndwi]]
    combined = tf.keras.layers.Concatenate()(branches)

    x = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same')(combined)
    x = tf.keras.layers.MaxPooling2D(2,2)(x)
    x = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D(2,2)(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.AdamW(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------- Train Model -----------
model = create_model_with_indices(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# ----------- Evaluation -----------
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 64, 64, 6)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_13 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_14 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_15 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_16 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_17 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_18 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_15 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_13[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_16 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_14[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_17 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_15[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_18 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_16[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_19 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_17[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_20 (Conv2D)        │ (None, 64, 64, 32)     │            320 │ get_item_18[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_15          │ (None, 32, 32, 32)     │              0 │ conv2d_15[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_16          │ (None, 32, 32, 32)     │              0 │ conv2d_16[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_17          │ (None, 32, 32, 32)     │              0 │ conv2d_17[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_18          │ (None, 32, 32, 32)     │              0 │ conv2d_18[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├──────────────────────

 Total params: 1,236,426 (4.72 MB)

 Trainable params: 1,236,426 (4.72 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - accuracy: 0.5727 - loss: 1.2530 - val_accuracy: 0.8383 - val_loss: 0.4644
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8590 - loss: 0.4076 - val_accuracy: 0.8930 - val_loss: 0.3096
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8946 - loss: 0.3007 - val_accuracy: 0.8931 - val_loss: 0.3102
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9131 - loss: 0.2472 - val_accuracy: 0.9044 - val_loss: 0.2736
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9315 - loss: 0.1915 - val_accuracy: 0.9106 - val_loss: 0.2512
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9447 - loss: 0.1478 - val_accuracy: 0.8874 - val_loss: 0.3689
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9517 - loss: 0.1276 - val_accuracy: 0.9348 - val_loss: 0.2011
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9643 - loss: 0.1074 -

**PROPOSED SE BASED NDBI-NIR-RED**

In [11]:
import os
import numpy as np
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# --- Load and Preprocess ---
def load_index_images(base_dir, img_size=(64, 64)):
    images, labels = [], []
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    
    for label in class_names:
        class_dir = os.path.join(base_dir, label)
        for fname in os.listdir(class_dir):
            if fname.endswith(".tif"):
                path = os.path.join(class_dir, fname)
                with rasterio.open(path) as src:
                    B3 = src.read(3).astype(np.float32)
                    B4 = src.read(4).astype(np.float32)
                    B5 = src.read(5).astype(np.float32)
                    B6 = src.read(6).astype(np.float32)

                    NDVI = (B5 - B4) / (B5 + B4 + 1e-5)
                    NDBI = (B6 - B5) / (B6 + B5 + 1e-5)
                    NDWI = (B3 - B5) / (B3 + B5 + 1e-5)

                    index_img = np.stack([NDBI, NDVI, NDWI], axis=-1)
                    index_img = np.nan_to_num(index_img)
                    index_img = tf.image.resize(index_img, img_size).numpy()
                    index_img = np.clip(index_img, -1, 1)
                    images.append(index_img)
                    labels.append(label)

    return np.array(images), np.array(labels), class_names

data_dir = "/kaggle/input/all-band-eurosat/tif"
X, y, class_names = load_index_images(data_dir)

# --- Label Encoding ---
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)

# --- Train-Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, stratify=y_encoded, random_state=42)
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)

# --- SE Block ---
def se_block(input_tensor, reduction=8):
    filters = input_tensor.shape[-1]
    se = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
    se = tf.keras.layers.Dense(filters // reduction, activation='relu')(se)
    se = tf.keras.layers.Dense(filters, activation='sigmoid')(se)
    se = tf.keras.layers.Reshape((1, 1, filters))(se)
    return tf.keras.layers.Multiply()([input_tensor, se])

# --- Model Definition (~1.5M Params) ---
def create_index_model(num_classes):
    inputs = tf.keras.Input(shape=(64, 64, 3))
    
    ch1 = tf.keras.layers.Conv2D(24, 3, activation='relu', padding='same')(inputs[:, :, :, 0:1])
    ch1 = se_block(ch1)
    ch1 = tf.keras.layers.MaxPooling2D()(ch1)
    
    ch2 = tf.keras.layers.Conv2D(24, 3, activation='relu', padding='same')(inputs[:, :, :, 1:2])
    ch2 = se_block(ch2)
    ch2 = tf.keras.layers.MaxPooling2D()(ch2)
    
    ch3 = tf.keras.layers.Conv2D(24, 3, activation='relu', padding='same')(inputs[:, :, :, 2:2+1])
    ch3 = se_block(ch3)
    ch3 = tf.keras.layers.MaxPooling2D()(ch3)
    
    merged = tf.keras.layers.Concatenate()([ch1, ch2, ch3])
    
    x = tf.keras.layers.Conv2D(48, 3, activation='relu', padding='same')(merged)
    x = se_block(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    
    x = tf.keras.layers.Conv2D(96, 3, activation='relu', padding='same')(x)
    x = se_block(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(192, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.AdamW(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- Train ---
model = create_index_model(num_classes=len(class_names))
model.summary()
model.fit(train_ds, validation_data=test_ds, epochs=100)

# --- Evaluation ---
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(np.vstack([y for _, y in test_ds]), axis=1)

cm = confusion_matrix(y_true, y_pred)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

print(f"\n{'Class':<20} {'Accuracy':<10}")
for i, label in enumerate(class_names):
    print(f"{label:<20} {per_class_acc[i]:.5f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=5))


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_37 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_38 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_39 (GetItem)     │ (None, 64, 64, 1)      │              0 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_63 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_37[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_64 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_38[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_65 (Conv2D)        │ (None, 64, 64, 24)     │            240 │ get_item_39[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_63[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_64[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 24)             │              0 │ conv2d_65[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_70 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_72 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_74 (Dense)          │ (None, 3)              │             75 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_71 (Dense)          │ (None, 24)             │             96 │ dense_70[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_73 (Dense)          │ (None, 24)             │             96 │ dense_72[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_75 (Dense)          │ (None, 24)             │             96 │ dense_74[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_25 (Reshape)      │ (None, 1, 1, 24)       │              0 │ dense_71[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_26 (Reshape) 

 Total params: 1,258,765 (4.80 MB)

 Trainable params: 1,258,765 (4.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.5606 - loss: 1.1884 - val_accuracy: 0.8111 - val_loss: 0.5377
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8217 - loss: 0.5087 - val_accuracy: 0.8222 - val_loss: 0.5129
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8605 - loss: 0.4062 - val_accuracy: 0.8620 - val_loss: 0.4032
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8856 - loss: 0.3384 - val_accuracy: 0.8915 - val_loss: 0.3264
Epoch 5/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8895 - loss: 0.3158 - val_accuracy: 0.8952 - val_loss: 0.3061
Epoch 6/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9080 - loss: 0.2729 - val_accuracy: 0.9050 - val_loss: 0.2767
Epoch 7/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9120 - loss: 0.2526 - val_accuracy: 0.8961 - val_loss: 0.3104
Epoch 8/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9246 - loss: 0.2131 -